In [ ]:
import os

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import bilby
import corner
import h5py
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import glob
import numpy as np
import pandas as pd

from importlib import reload
import thesis_utils
from thesis_utils.gw import get_cbc_parameter_labels

reload(thesis_utils.gw.utils)

from thesis_utils.plotting import (
    get_default_corner_kwargs,
    set_plotting,
    get_default_figsize,
    save_figure,
)


set_plotting()

In [ ]:
gwtc2p1_file = "/home/michael.williams/git_repos/nessai-gwtc-1/gwtc-2.1_sample_release/IGWN-GWTC2p1-v2-GW170823_131358_PEDataRelease_mixed_nocosmo.h5"
with h5py.File(gwtc2p1_file, "r") as f:
    print(f.keys())
    gwtc2p1_result = f["C01:IMRPhenomXPHM/posterior_samples"][()]

In [ ]:
inessai_xphm = bilby.core.result.read_in_result(
    "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/inessai/outdir_inessai_gwtc_1_GW170823_cal_reweight_XPHM_v7/final_result/inessai_gwtc_1_GW170823_XPHM_data0_1187529256-517903_analysis_H1L1_merge_result.hdf5"
)

In [ ]:
nessai_xphm = bilby.core.result.read_in_result(
    "/home/michael.williams/git_repos/nessai-gwtc-1/analysis/IMRPhenomXPHM/outdir_nessai_gwtc_1_GW170823_cal_reweight_XPHM/final_result/nessai_gwtc_1_GW170823_XPHM_data0_1187529256-517903_analysis_H1L1_merge_result.hdf5"
)

In [ ]:
parameters = ["chirp_mass", "mass_ratio", "chi_p", "chi_eff", "ra", "dec", "theta_jn"]

corner_kwargs = get_default_corner_kwargs()
corner_kwargs["plot_density"] = True
corner_kwargs["plot_datapoints"] = False
corner_kwargs["no_fill_contours"] = True
corner_kwargs["show_titles"] = False
corner_kwargs.pop("fill_contours")
corner_kwargs["levels"] = (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.0))
corner_kwargs["quantiles"] = []
corner_kwargs["smooth"] = 0.9
corner_kwargs["bins"] = 32
figsize = 1.8 * get_default_figsize()
figsize[1] = figsize[0]
fig = plt.figure(figsize=figsize)

corner_kwargs["color"] = "k"
corner_kwargs["hist_kwargs"]["color"] = "k"
corner_kwargs["hist_kwargs"]["ls"] = "--"
fig = corner.corner(
    np.array([gwtc2p1_result[p] for p in parameters]).T,
    fig=fig,
    contour_kwargs=dict(linestyles=["--"]),
    **corner_kwargs,
)

corner_kwargs["hist_kwargs"]["ls"] = "-"
corner_kwargs["color"] = "C0"
corner_kwargs["hist_kwargs"]["color"] = "C0"
fig = corner.corner(
    np.array([nessai_xphm.posterior[p] for p in parameters]).T,
    fig=fig,
    labels=get_cbc_parameter_labels(parameters, units=True),
    labelpad=-0.1,
    **corner_kwargs,
)

corner_kwargs["hist_kwargs"]["ls"] = "-"
corner_kwargs["color"] = "C1"
corner_kwargs["hist_kwargs"]["color"] = "C1"
fig = corner.corner(
    np.array([inessai_xphm.posterior[p] for p in parameters]).T,
    fig=fig,
    labels=get_cbc_parameter_labels(parameters, units=True),
    labelpad=-0.1,
    **corner_kwargs,
)

legend_elements = [
    Line2D([0], [0], color="k", ls="--", label=r"GWTC-2.1"),
    Line2D([0], [0], color="C0", ls="-", label=r"\texttt{nessai}"),
    Line2D([0], [0], color="C1", ls="-", label=r"\texttt{i-nessai}"),
]

fig.legend(
    handles=legend_elements,
    bbox_to_anchor=(1.0, 0.9),
    loc="upper right",
    fontsize=18,
)
plt.show()

save_figure(fig, "GW170823_posterior_XPHM", "figures/inessai/")